# 옵서버 패턴

## 행위 패턴 소개

- 생성 패턴은 객체가 생성되는 방식을 다룸
- 구조 패턴은 객체와 클래스를 합쳐 더 큰 기능을 구현하는 방식을 다룸
- 행위 패턴
  - 객체의 역할에 점을 둠
  - 객체는 상호 작용하지만, 느슨하게 결합함

## 목적

- 객체 간 일대다(1:N) 관계를 형성하고 객체의 상태를 다른 종속 객체에 자동으로 알림
- 서브젝트의 핵심 부분을 캡슐화

## 적용 사례

- 분산 시스템의 이벤트 서비스 구현
- 뉴스 에이전시 프레임워크
- 주식 시장 모델

## 옵서버 패턴 메소드

### 풀(Pull) 모델

풀 모델에서 Observer의 역할

- Subject는 변경 사항 발생시 등록된 Observer에 브로드캐스트
- Observer는 직접 게시자에게 변경 사항을 요청하고 끌어옴(pull)
- Subject가 Observer에 알리는 단계와 Observer가 Subject로부터 데이터를 받아오는 두 단계가 필요하므로 비효율적

### 푸시(Push) 모델

푸시 모델에서 Subject의 역할

- 풀 모델과 달리 Subject가 Observer에 데이터를 전송
- Subject는 Observer가 필요로 하지 않는 데이터까지 보낼 수 있음
- 쓸데없이 많은 양의 데이터를 전송해 응답 시간이 늦어질 수 있음
- 성능을 위해 Subject는 오직 필요한 데이터만 보내야 함

## 느슨한 결합(Loose coupling)

### 느슨한 결합의 효과

- 한 부분의 수정이 예기치 않게 다른 부분까지 영향을 끼치는 위험을 줄임
- 테스트와 유지 보수, 장애 처리가 쉬움
- 시스템을 쉽게 여러 부분으로 분리

### 옵서버 패턴에서의 느슨한 결합

- Subject는 정확히 Observer가 어떤 인터페이스를 구현하는지 모름. ConcreteObserver의 존재를 모름
- 언제든 새로운 Observer 추가 가능
- 새로운 Observer를 추가해도 Subject를 수정할 필요가 없음
- Subject 또는 Observer는 독립적임. Observer는 필요시 어디에서도 재사용 가능
- Subject 또는 Observer에 대한 수정이 서로에게 아무 영향을 주지 않음

## 옵서버 패턴의 장단점

### 장점

- 객체 간의 느슨한 결합 원칙을 따름
- Subject 또는 Observer 클래스를 수정하지 않고 객체 간 자유롭게 데이터를 주고 받을 수 있음
- 새로운 Observer를 언제든지 추가/제거할 수 있음

### 단점

- ConcreteObserver는 상속을 통해 Observer 인터페이스를 구현. 컴포지션에 대한 선택권이 없음
- 제대로 구현하지 못한 Observer 클래스는 복잡도를 높이고 성능 저하의 원인이 될 수 있음
- 애플리케이션에서 알림(Notification) 기능은 간혹 신뢰할 수 없으며 레이스 상태(Race Condition) 또는 비일관성을 초래할 수 있음

In [5]:
# 옵서버 디자인 패턴
class Subject:
    def __init__(self):
        self.__observers = []

    def register(self, observer):
        self.__observers.append(observer)

    def notifyAll(self, *args, **kwargs):
        for observer in self.__observers:
            observer.notify(self, *args, **kwargs)

class Observer1:
    def __init__(self, subject):
        subject.register(self)

    def notify(self, subject, *args):
        print(type(self).__name__, ":: Got", args, "from", subject)

class Observer2:
    def __init__(self, subject):
        subject.register(self)

    def notify(self, subject, *args):
        print(type(self).__name__, ":: Got", args, "from", subject)

subject = Subject()
observer1 = Observer1(subject)
observer2 = Observer2(subject)
subject.notifyAll("notification")

# Observer1 :: Got ('notification',) from <__main__.Subject object at 0x10a3b0950>
# Observer2 :: Got ('notification',) from <__main__.Subject object at 0x10a3b0950>

Observer1 :: Got ('notification',) from <__main__.Subject object at 0x10a3b0950>
Observer2 :: Got ('notification',) from <__main__.Subject object at 0x10a3b0950>


In [6]:
# 옵서버 패턴 - 뉴스 에이전시
from abc import ABCMeta, abstractmethod

class NewsPublisher:
    def __init__(self):
        self.__subscribers = []
        self.__latestNews = None

    def attach(self, subscriber):
        self.__subscribers.append(subscriber)

    def detach(self):
        return self.__subscribers.pop()

    def subscribers(self):
        return [type(x).__name__ for x in self.__subscribers]

    def notifySubscribers(self):
        for sub in self.__subscribers:
            sub.update()

    def addNews(self, news):
        self.__latestNews = news

    def getNews(self):
        return "Got News:", self.__latestNews

class Subscriber(metaclass=ABCMeta):

    @abstractmethod
    def update(self):
        pass

class SMSSubcriber(Subscriber):
    def __init__(self, publisher):
        self.publisher = publisher
        self.publisher.attach(self)

    def update(self):
        print(type(self).__name__, self.publisher.getNews())

class EmailSubcriber(Subscriber):
    def __init__(self, publisher):
        self.publisher = publisher
        self.publisher.attach(self)

    def update(self):
        print(type(self).__name__, self.publisher.getNews())

class AnotherSubcriber(Subscriber):
    def __init__(self, publisher):
        self.publisher = publisher
        self.publisher.attach(self)

    def update(self):
        print(type(self).__name__, self.publisher.getNews())

if __name__ == "__main__":
    news_publisher = NewsPublisher()
    for Subscribers in [SMSSubcriber, EmailSubcriber, AnotherSubcriber]:
        Subscribers(news_publisher)
    print("\nSubscribers:", news_publisher.subscribers())

    news_publisher.addNews("Hello World!")
    news_publisher.notifySubscribers()

    print("\nDatached:", type(news_publisher.detach()).__name__)
    print("\nSubscribers:", news_publisher.subscribers())

    news_publisher.addNews("My second news!")
    news_publisher.notifySubscribers()

# Subscribers: ['SMSSubcriber', 'EmailSubcriber', 'AnotherSubcriber']
# SMSSubcriber ('Got News:', 'Hello World!')
# EmailSubcriber ('Got News:', 'Hello World!')
# AnotherSubcriber ('Got News:', 'Hello World!')

# Datached: AnotherSubcriber

# Subscribers: ['SMSSubcriber', 'EmailSubcriber']
# SMSSubcriber ('Got News:', 'My second news!')
# EmailSubcriber ('Got News:', 'My second news!')


Subscribers: ['SMSSubcriber', 'EmailSubcriber', 'AnotherSubcriber']
SMSSubcriber ('Got News:', 'Hello World!')
EmailSubcriber ('Got News:', 'Hello World!')
AnotherSubcriber ('Got News:', 'Hello World!')

Datached: AnotherSubcriber

Subscribers: ['SMSSubcriber', 'EmailSubcriber']
SMSSubcriber ('Got News:', 'My second news!')
EmailSubcriber ('Got News:', 'My second news!')
